#!/usr/bin/env python3

This is a script to calculate accumulated precipitation time series by calibration catchment.
* Precip is accumulated in time and on the catchments area.
* Output is in [precip unit]/[area unit] i.e. mm/km2
* Accumulation period is selected with a flag.

The scrip can also do rolling accumulation over the entire period.

Note: hard coded variables names "Band1" "pixarea" "pr6"

Usage:

```ts_from_meteomaps.py  -i /path_to/pr6.nc -I /path_to/catchments.nc -O /path_to/output.csv -t sum_month -s 201701011200 -e 201712311200 -A /path_to/pixarea.nc```

In [2]:
import xarray as xr
import pandas as pd
import numpy as np
import datetime
import os.path

In [2]:
def getarg():
    """ Get program arguments.

    :return: args:  namespace of program arguments
    """
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument('-i', '--datadir', type=str, required=True,
                        help='data folder')
    parser.add_argument('-I', '--interbasins', type=str, required=True,
                        help='Interbasins file')
    parser.add_argument('-O', '--outputfilename', type=str, required=False, default="indexes.csv",
                        help='Output file')
    parser.add_argument('-s', '--startdate', type=str, required=True,
                        help='Start date for evaluation [YYYYMMDDhhmm]')
    parser.add_argument('-e', '--enddate', type=str, required=True,
                        help='End date for evaluation [YYYYMMDDhhmm]')
    parser.add_argument('-t', '--ctype', type=str, required=False, default='All',
                        help='Aggregation type [sum_all|sum_year|ts]')
    parser.add_argument('-A', '--interbasins_area', type=str, required=True,
                        help='Interbasins area file')
    args = parser.parse_args()  # assign namespace to args
    return args

In [2]:
def extract_time_series(pr6, ibasins, area):
    """ Calc time series of accumulated values for catchment areas (ibasins) from DataArray (pr).

        Put the time series in a DataFrame using time as index and catchment code as header.

    :return: tssdata: pandas.DataFrame with time series of accumulated values by catchment
    """
    # Calc accumulated values over catchments
    area_tot_group = area.groupby(ibasins).sum()    #[m2]
    # Put in dataframe
    area_tot_df = area_tot_group.to_dataframe()     #[m2]

    # calc volume for each cell
    pr = pr6 * area  #[m3]
    # group cells in pr DataArray by catchment ID (ibasins)
    pr6_ts_group = pr.groupby(ibasins)  #[m3]
    # sum over catchments ID
    #pr6_ts = pr6_ts_group.sum(dim='stacked_y_x')    #[m3]
    pr6_ts = pr6_ts_group.sum(dim='stacked_lat_lon')    #[m3]
    # put in multi-index dataframe
    pr6_ts_df = pr6_ts.to_dataframe(name='pr6')       #[m3]
    # divide by catchments area
    #pr6_ts_df['pr6'] = pr6_ts_df['pr6'].div(area_tot_df['pixarea'].values,axis=0)   #[m]

    # generate single index dataframe and store time series
    dates = pr6_ts_df.index.unique(level="time").values
    catcodes = pr6_ts_df.index.unique(level="ibasins").values
    tssdata = pd.DataFrame(index = dates)
    for code in catcodes:
        # print(code)
        # extract ts for one catchment from multi-*index dataframe
        ts = pr6_ts_df.xs(code, level='ibasins', drop_level=True)     #[m3]
        # average by catchment area
        ts_avg = pd.DataFrame()
        ts_avg['pr6'] = ts['pr6'] / area_tot_df.loc[code,'pixarea'] * 1000.     #[mm]
        # use code as header
        ts_avg = ts_avg.rename(columns={'pr6': str(int(code))})
        # store in dataframe
        tssdata = pd.merge(tssdata,ts_avg,how='outer',left_index=True,right_index=True, copy=False)
    return tssdata

In [2]:
def extract_total_accumulation(pr6, ibasins, area):
    """ Calc accumulated values for catchment areas (ibasins) from DataArray (pr) over entire time period.

        Put accumulated values in a DataFrame with ObsID as index.

    :return: tssdata: pandas.DataFrame with accumulated values by catchment
    """
    # Calc accumulated precip volume over time
    pr6_sum = pr6.sum(dim='time')   #[m]
    pr6_tot = pr6_sum * area  #[m3]

    # Calc accumulated values over catchments
    pr6_tot_group = pr6_tot.groupby(ibasins).sum()  #[m3]
    area_tot_group = area.groupby(ibasins).sum()    #[m2]

    # Put in dataframe
    pr6_tot_df = pr6_tot_group.to_dataframe(name='pr6')
    area_tot_df = area_tot_group.to_dataframe()

    # Divide precip volume [m3] by catchment area [m2] and transform to [mm]
    pr6_tot_df['pr6'] = pr6_tot_df['pr6'].div(area_tot_df['pixarea'].values,axis=0) * 1000.  #[mm]

    # rename index
    pr6_tot_df.index.name = 'ObsID'
    # set index to integer
    pr6_tot_df.index = pr6_tot_df.index.astype('int', copy=False)
    # rename column
    pr6_tot_df.rename(columns={'pr6': 'pr6_tot'})
    return pr6_tot_df

In [2]:
#-- settings
# datadir = '/perm/mo/mocm/proj/efas/hprot/GRIDS_5x5km_efas4_dataset_new_sheremap/'
# interbasinsfile = '/perm/mo/mocm/proj/efas/hres/meteo_test/pcraster_scripts/efas4_cats.nc'
# outputfile = '/perm/mo/mocm/proj/efas/hres/meteo_test/ef4_dataset_new_sheremap_ts.csv'

def main():
    # read settings
    args = getarg()
    datadir = args.datadir
    interbasinsfile= args.interbasins
    areafile = args.interbasins_area
    outputfile = args.outputfilename
    startdate = datetime.datetime.strptime(args.startdate, '%Y%m%d%H%M')
    enddate = datetime.datetime.strptime(args.enddate, '%Y%m%d%H%M')
    flag_type = args.ctype

    ### Read inputs
    # Get precip
    extension = os.path.splitext(datadir)[1]
    if extension == '.nc':
        # get from one single .nc file
        fdata = datadir
        DS = xr.open_dataset(fdata, chunks={'time': 'auto'})
    else:
        # Get forcings from multiple nc files
        mfdataDIR = datadir + 'pr6_*'
        DS = xr.open_mfdataset(mfdataDIR, chunks={'time': 'auto'})
    for name in list(DS.data_vars):
        if name == 'precip':
            DS = DS.rename({"precip": "pr6"})

    # Get cell area map
    DS_area = xr.open_dataset(areafile)
    # Sort Lat (just in case)
    DS_area = DS_area.sortby('lat', ascending=False)
    for name in list(DS_area.data_vars):
        if name == 'Band1':
            DS_area = DS_area.rename({"Band1": "pixarea"})
    area = DS_area.pixarea  #[m2]
    #area = area / area  # this is to basically remove area (:D)
    # put in numpy array
    areanp = area.values
    # assign to dataset
    DS["pixarea"] = (['lat','lon'],  areanp)

    # Get catchments masks from nc file
    DS_ibasins = xr.open_dataset(interbasinsfile)
    # Flip Lat coord because map comes from pcraster
    DS_ibasins = DS_ibasins.sortby('lat', ascending=False)
    ibasins = DS_ibasins.Band1
    ibasins = ibasins.where(ibasins != 0)
    # put it in numpy array
    ibasinsnp = ibasins.values
    # assign to dataset
    #DS["ibasins"]=(['y','x'],  ibasinsnp)
    DS["ibasins"]=(['lat','lon'],  ibasinsnp)

    # Extract variable pr6 from array.Dataset for the selected period and put in xarray.DataArray
    pr6 = DS.pr6.sel(time=slice(startdate, enddate)) * 0.25 / 1000.     #input data in mm/day and Dt=6hrs -> mm -> m
    pixarea = DS.pixarea
    ibas = DS.ibasins

    # # Calc accumulated values over catchments
    # area_tot_group = pixarea.groupby(ibas).sum()    #[m2]
    # # Put in dataframe
    # area_tot_df = area_tot_group.to_dataframe() / 1000000.  #[km2]

    # Calc accumulated values by catchment with sum pr6 values over full period
    if flag_type == 'sum_tot' :
        tssdata = extract_total_accumulation(pr6,ibas,pixarea)
        print(flag_type)

    # Calc accumulated values by catchment with sum pr6 values over years
    # Calc yearly time series accumulated by catchment
    if flag_type == 'sum_year':
        # accumulate by year
        pr = pr6.resample(time="Y").sum()
        tssdata = extract_time_series(pr,ibas,pixarea)
        print(flag_type)

    if flag_type == 'sumcum_year':
        # accumulate by year
        pr = pr6.resample(time="Y").sum()
        tssdata = extract_time_series(pr,ibas,pixarea)
        tssdata = tssdata.cumsum(axis='index')
        print(flag_type)

    # Calc monthly time series accumulated by catchment
    if flag_type == 'sum_month':
        # accumulate by month
        pr = pr6.resample(time="M").sum()
        tssdata = extract_time_series(pr,ibas,pixarea)
        print(flag_type)

    if flag_type == 'sumcum_month':
        # accumulate by month
        pr = pr6.resample(time="M").sum()
        tssdata = extract_time_series(pr,ibas,pixarea)
        tssdata = tssdata.cumsum(axis='index')
        print(flag_type)

    # Calc time series accumulated by catchment
    if flag_type == 'ts':
        # no accumulation
        # pr = pr6
        tssdata = extract_time_series(pr6,ibas,pixarea)
        print(flag_type)

    if flag_type == 'sumcum_ts':
        # no accumulation
        # pr = pr6
        tssdata = extract_time_series(pr6,ibas,pixarea)
        tssdata = tssdata.cumsum(axis='index')
        print(flag_type)

    # print results TO CSV FILE
    with open(outputfile, 'w') as outf:
        tssdata.to_csv(outf,sep=',')

    # # print results TO CSV FILE
    # with open('/perm/mo/mocm/proj/efas/hres/meteo_test/interpolation/sph_2018-08-29_ef4_1min/area_bas_efas5.csv', 'w') as outf:
    #     area_tot_df.to_csv(outf,sep=',')


if __name__ == "__main__":
    main()

SyntaxError: invalid or missing encoding declaration for 'ts_from_meteomaps.py' (<string>)